https://sanjayasubedi.com.np/python/sqlalchemy/recursive-query-in-postgresql-with-sqlalchemy/

In [1]:
import os

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy import Column, Integer, String, ForeignKey

In [3]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [4]:
Base = declarative_base()

In [5]:
class Project(Base):
    __tablename__ = 'projects'
    id = Column(Integer, primary_key=True)
    project_name = Column(String)
    full_path = Column(String)
    parent_id = Column(Integer)

In [6]:
class Job(Base):
    __tablename__ = 'jobs'
    id = Column(Integer, primary_key=True)
    job_name = Column(String)
    master_id = Column(Integer)
    parent_id = Column(Integer)
    project_id = Column(Integer, ForeignKey("projects.id"))

In [7]:
Base.metadata.create_all(engine)

2022-03-13 16:31:10,044 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-13 16:31:10,045 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("projects")
2022-03-13 16:31:10,045 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-13 16:31:10,046 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("projects")
2022-03-13 16:31:10,046 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-13 16:31:10,046 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("jobs")
2022-03-13 16:31:10,046 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-13 16:31:10,047 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("jobs")
2022-03-13 16:31:10,047 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-13 16:31:10,048 INFO sqlalchemy.engine.Engine 
CREATE TABLE projects (
	id INTEGER NOT NULL, 
	project_name VARCHAR, 
	full_path VARCHAR, 
	parent_id INTEGER, 
	PRIMARY KEY (id)
)


2022-03-13 16:31:10,048 INFO sqlalchemy.engine.Engine [no key 0.00024s] ()
2022-03-13 16:31:10,049 INFO sqlalchemy.engine.E

In [8]:
session = sessionmaker(bind=engine)()

In [9]:
def get_path_dict(path):
    cwd_split_lst = path.split("/")
    return {
        n:p for n, p in zip(
            [p if p != "" else "/" for p in cwd_split_lst], 
            ["/".join(cwd_split_lst[:i+1]) if i > 0 else "/" for i in range(len(cwd_split_lst))]
        )
    }

In [10]:
def get_project_id(session, full_path):
    result = session.query(Project).filter(Project.full_path==full_path).all()
    if len(result) > 0:
        return result[0]
    else:
        return None

In [11]:
def get_path_to_add(sub_path_dict):
    full_path_lst = list(sub_path_dict.values())[::-1]
    project_name_lst = list(sub_path_dict.keys())[::-1]
    project_path_lst = [get_project_id(session=session, full_path=p) for p in full_path_lst]
    project_id_lst = [p for p in project_path_lst if p is not None]
    if len(project_id_lst) == 0:
        return sub_path_dict, None
    else:
        return {
            k: v 
            for k, v in sub_path_dict.items() 
            if project_id_lst[0].full_path in v and not project_id_lst[0].full_path == v
        }, project_id_lst[0]

In [12]:
def add_path_to_database(path_dict, project_prev=None):
    for project_name, full_path in path_dict.items(): 
        if project_prev is None:
            project_cur = Project(
                project_name=project_name, 
                full_path=full_path
            )
        else:
            project_cur = Project(
                project_name=project_name, 
                full_path=full_path, 
                parent_id=project_prev.id
            )
        session.add(project_cur)
        session.commit()
        project_prev = project_cur

In [13]:
def get_project_ids(directory, recursive=False):
    parent_project = session.query(Project).filter(Project.full_path==directory).one()
    if not recursive:
        return [parent_project.id]
    else: 
        topq = session.query(Project)
        topq = topq.filter(Project.id == parent_project.id)
        topq = topq.cte('cte', recursive=True)
        
        bottomq = session.query(Project)
        bottomq = bottomq.join(topq, Project.parent_id == topq.c.id)
        
        recursive_q = topq.union(bottomq)
        result_lst = session.query(recursive_q).all()
        return [r.id for r in result_lst]

In [14]:
cwd = os.path.abspath(".")

In [15]:
sub_path_dict = get_path_dict(path=os.path.dirname(os.path.dirname(cwd)))
sub_path_dict

{'/': '/',
 'Users': '/Users',
 'jan': '/Users/jan',
 'pyiron': '/Users/jan/pyiron',
 'projects': '/Users/jan/pyiron/projects'}

In [16]:
sub_path_update_dict, project_prev = get_path_to_add(sub_path_dict=sub_path_dict)
sub_path_update_dict, project_prev

2022-03-13 16:31:10,077 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-13 16:31:10,079 INFO sqlalchemy.engine.Engine SELECT projects.id AS projects_id, projects.project_name AS projects_project_name, projects.full_path AS projects_full_path, projects.parent_id AS projects_parent_id 
FROM projects 
WHERE projects.full_path = ?
2022-03-13 16:31:10,079 INFO sqlalchemy.engine.Engine [generated in 0.00028s] ('/Users/jan/pyiron/projects',)
2022-03-13 16:31:10,080 INFO sqlalchemy.engine.Engine SELECT projects.id AS projects_id, projects.project_name AS projects_project_name, projects.full_path AS projects_full_path, projects.parent_id AS projects_parent_id 
FROM projects 
WHERE projects.full_path = ?
2022-03-13 16:31:10,080 INFO sqlalchemy.engine.Engine [cached since 0.001069s ago] ('/Users/jan/pyiron',)
2022-03-13 16:31:10,081 INFO sqlalchemy.engine.Engine SELECT projects.id AS projects_id, projects.project_name AS projects_project_name, projects.full_path AS projects_full_path, proj

({'/': '/',
  'Users': '/Users',
  'jan': '/Users/jan',
  'pyiron': '/Users/jan/pyiron',
  'projects': '/Users/jan/pyiron/projects'},
 None)

In [17]:
add_path_to_database(path_dict=sub_path_update_dict, project_prev=project_prev)

2022-03-13 16:31:10,085 INFO sqlalchemy.engine.Engine INSERT INTO projects (project_name, full_path, parent_id) VALUES (?, ?, ?)
2022-03-13 16:31:10,086 INFO sqlalchemy.engine.Engine [generated in 0.00049s] ('/', '/', None)
2022-03-13 16:31:10,086 INFO sqlalchemy.engine.Engine COMMIT
2022-03-13 16:31:10,087 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-13 16:31:10,088 INFO sqlalchemy.engine.Engine SELECT projects.id AS projects_id, projects.project_name AS projects_project_name, projects.full_path AS projects_full_path, projects.parent_id AS projects_parent_id 
FROM projects 
WHERE projects.id = ?
2022-03-13 16:31:10,088 INFO sqlalchemy.engine.Engine [generated in 0.00025s] (1,)
2022-03-13 16:31:10,089 INFO sqlalchemy.engine.Engine INSERT INTO projects (project_name, full_path, parent_id) VALUES (?, ?, ?)
2022-03-13 16:31:10,089 INFO sqlalchemy.engine.Engine [cached since 0.003511s ago] ('Users', '/Users', 1)
2022-03-13 16:31:10,089 INFO sqlalchemy.engine.Engine COMMIT
2022-03

In [18]:
sub_path_dict = get_path_dict(path=os.path.abspath("."))
sub_path_dict

{'/': '/',
 'Users': '/Users',
 'jan': '/Users/jan',
 'pyiron': '/Users/jan/pyiron',
 'projects': '/Users/jan/pyiron/projects',
 '2022': '/Users/jan/pyiron/projects/2022',
 '2022-03-13-project-sql': '/Users/jan/pyiron/projects/2022/2022-03-13-project-sql'}

In [19]:
sub_path_update_dict, project_prev = get_path_to_add(sub_path_dict=sub_path_dict)
sub_path_update_dict, project_prev

2022-03-13 16:31:10,100 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-13 16:31:10,101 INFO sqlalchemy.engine.Engine SELECT projects.id AS projects_id, projects.project_name AS projects_project_name, projects.full_path AS projects_full_path, projects.parent_id AS projects_parent_id 
FROM projects 
WHERE projects.full_path = ?
2022-03-13 16:31:10,101 INFO sqlalchemy.engine.Engine [cached since 0.02216s ago] ('/Users/jan/pyiron/projects/2022/2022-03-13-project-sql',)
2022-03-13 16:31:10,102 INFO sqlalchemy.engine.Engine SELECT projects.id AS projects_id, projects.project_name AS projects_project_name, projects.full_path AS projects_full_path, projects.parent_id AS projects_parent_id 
FROM projects 
WHERE projects.full_path = ?
2022-03-13 16:31:10,102 INFO sqlalchemy.engine.Engine [cached since 0.02287s ago] ('/Users/jan/pyiron/projects/2022',)
2022-03-13 16:31:10,102 INFO sqlalchemy.engine.Engine SELECT projects.id AS projects_id, projects.project_name AS projects_project_name, p

({'2022': '/Users/jan/pyiron/projects/2022',
  '2022-03-13-project-sql': '/Users/jan/pyiron/projects/2022/2022-03-13-project-sql'},
 <__main__.Project at 0x111cae650>)

In [20]:
add_path_to_database(path_dict=sub_path_update_dict, project_prev=project_prev)

2022-03-13 16:31:10,108 INFO sqlalchemy.engine.Engine INSERT INTO projects (project_name, full_path, parent_id) VALUES (?, ?, ?)
2022-03-13 16:31:10,108 INFO sqlalchemy.engine.Engine [cached since 0.02301s ago] ('2022', '/Users/jan/pyiron/projects/2022', 5)
2022-03-13 16:31:10,109 INFO sqlalchemy.engine.Engine COMMIT
2022-03-13 16:31:10,109 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-13 16:31:10,110 INFO sqlalchemy.engine.Engine SELECT projects.id AS projects_id, projects.project_name AS projects_project_name, projects.full_path AS projects_full_path, projects.parent_id AS projects_parent_id 
FROM projects 
WHERE projects.id = ?
2022-03-13 16:31:10,110 INFO sqlalchemy.engine.Engine [cached since 0.02198s ago] (6,)
2022-03-13 16:31:10,110 INFO sqlalchemy.engine.Engine INSERT INTO projects (project_name, full_path, parent_id) VALUES (?, ?, ?)
2022-03-13 16:31:10,110 INFO sqlalchemy.engine.Engine [cached since 0.02506s ago] ('2022-03-13-project-sql', '/Users/jan/pyiron/projects

In [21]:
parent_dir = os.path.dirname(os.path.dirname(cwd))

In [22]:
get_project_ids(directory=parent_dir, recursive=False)

2022-03-13 16:31:10,115 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-13 16:31:10,116 INFO sqlalchemy.engine.Engine SELECT projects.id AS projects_id, projects.project_name AS projects_project_name, projects.full_path AS projects_full_path, projects.parent_id AS projects_parent_id 
FROM projects 
WHERE projects.full_path = ?
2022-03-13 16:31:10,116 INFO sqlalchemy.engine.Engine [cached since 0.0372s ago] ('/Users/jan/pyiron/projects',)


[5]

In [23]:
get_project_ids(directory=parent_dir, recursive=True)

2022-03-13 16:31:10,119 INFO sqlalchemy.engine.Engine SELECT projects.id AS projects_id, projects.project_name AS projects_project_name, projects.full_path AS projects_full_path, projects.parent_id AS projects_parent_id 
FROM projects 
WHERE projects.full_path = ?
2022-03-13 16:31:10,120 INFO sqlalchemy.engine.Engine [cached since 0.04066s ago] ('/Users/jan/pyiron/projects',)
2022-03-13 16:31:10,122 INFO sqlalchemy.engine.Engine WITH RECURSIVE cte(id, project_name, full_path, parent_id) AS 
(SELECT projects.id AS id, projects.project_name AS project_name, projects.full_path AS full_path, projects.parent_id AS parent_id 
FROM projects 
WHERE projects.id = ? UNION SELECT projects.id AS projects_id, projects.project_name AS projects_project_name, projects.full_path AS projects_full_path, projects.parent_id AS projects_parent_id 
FROM projects JOIN cte ON projects.parent_id = cte.id)
 SELECT cte.id AS cte_id, cte.project_name AS cte_project_name, cte.full_path AS cte_full_path, cte.parent_

[5, 6, 7]